In [ ]:
import os
import time
import random
import pandas as pd
import numpy as np
import time
import requests # type: ignore
import re
import logging
import subprocess
from subprocess import Popen
from sys import platform
import os, sys
import logging
import json
import threading
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import subprocess
import time
import signal
from typing import Dict

from netunicorn.base.architecture import Architecture # type: ignore
from netunicorn.base.nodes import Node # type: ignore
from netunicorn.base.task import Failure, Task, TaskDispatcher # type: ignore


from netunicorn.client.remote import RemoteClient, RemoteClientException # type: ignore
from netunicorn.base import Experiment, ExperimentStatus, Pipeline # type: ignore
from netunicorn.library.tasks.basic import SleepTask # type: ignore
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest # type: ignore
from netunicorn.library.tasks.measurements.ping import Ping # type: ignore
from netunicorn.base.architecture import Architecture # type: ignore
from netunicorn.base.nodes import Node # type: ignore
from netunicorn.base.task import Failure, Task, TaskDispatcher # type: ignore
from netunicorn.base import Result, Failure, Success, Task, TaskDispatcher # type: ignore
from netunicorn.base.architecture import Architecture # type: ignore
from netunicorn.base.nodes import Node # type: ignore
from netunicorn.base.pipeline import Pipeline # type: ignore
from netunicorn.library.tasks.basic import ShellCommand # type: ignore
from netunicorn.library.tasks.measurements.iperf3 import Iperf3ServerStart, Iperf3ServerStop, Iperf3Client # type: ignore
from netunicorn.library.tasks.measurements.ndt import _NDTSpeedTestImplementation, NDT7SpeedTest # type: ignore
from netunicorn.library.tasks.capture.tshark import StartCapture, StopCapture # type: ignore
from netunicorn.library.tasks.upload.fileio import UploadToFileIO # type: ignore
from netunicorn.library.tasks.upload.webdav import UploadToWebDav # type: ignore
from netunicorn.library.tasks.upload.fileio import UploadToFileIO, UploadToFileIOCurlImplementation # type: ignore


from typing import Dict
from typing import Optional
from enum import IntEnum
from datetime import datetime 

from returns.pipeline import is_successful # type: ignore
from returns.result import Failure # type: ignore

!pip install trustee

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from trustee import RegressionTrustee
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [11]:
def execute_pipeline(pipeline, working_node, experiment_label):
    client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD) # type: ignore
    nodes = client.get_nodes()
    working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)
    experiment = Experiment().map(pipeline, working_nodes)

    try:
        client.delete_experiment(experiment_label)
    except RemoteClientException:
        pass

    # Prepare Experiment
    client.prepare_experiment(experiment, experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status == ExperimentStatus.READY:
            break
        time.sleep(20)

    time.sleep(5)

    # Execute Experiment
    client.start_execution(experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status != ExperimentStatus.RUNNING:
            break
        time.sleep(20)
    return info

In [12]:
netunicorn_login = 'cs190n9'
netunicorn_password = '9lQERz4R'

NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', netunicorn_login)
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', netunicorn_password)

client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)

Health Check: True
[[snl-server-5], <Uncountable node pool with next node template: [aws-fargate-A-cs190n9-, aws-fargate-B-cs190n9-, aws-fargate-ARM64-cs190n9-]>]


In [13]:
working_nodes = 'aws-fargate-A-cs190n9-'

In [14]:
class ExtractInitialPacketFeaturesTask(Task):
    def __init__(self, pcap_file='capture.pcap', output_file='features.json', packet_count=10, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pcap_file = pcap_file
        self.output_file = output_file
        self.packet_count = packet_count

    def run(self):
        import pyshark
        import json

        capture = pyshark.FileCapture(self.pcap_file)
        features = []
        previous_timestamp = None

        for i, packet in enumerate(capture):
            if i >= self.packet_count:
                break

            feature = {
                'packet_size': int(packet.length),
                'timestamp': float(packet.sniff_timestamp),
                'protocol': packet.highest_layer,
            }

            # Extracting TCP features
            if 'TCP' in packet:
                tcp_layer = packet.tcp
                feature.update({
                    'src_port': int(tcp_layer.srcport),
                    'dst_port': int(tcp_layer.dstport),
                    'sequence_number': int(tcp_layer.seq),
                    'ack_number': int(tcp_layer.ack),
                    'window_size': int(tcp_layer.window_size_value),
                    'tcp_flags': tcp_layer.flags,
                })

            # Inter-packet arrival time
            if previous_timestamp is not None:
                feature['inter_arrival_time'] = feature['timestamp'] - previous_timestamp
            else:
                feature['inter_arrival_time'] = 0.0
            previous_timestamp = feature['timestamp']

            features.append(feature)

        capture.close()

        # Computing statistical features
        packet_sizes = [f['packet_size'] for f in features]
        inter_arrival_times = [f['inter_arrival_time'] for f in features[1:]]  

        statistical_features = {
            'mean_packet_size': sum(packet_sizes) / len(packet_sizes),
            'std_packet_size': (
                sum((x - sum(packet_sizes)/len(packet_sizes))**2 for x in packet_sizes) / len(packet_sizes)
            ) ** 0.5,
            'mean_inter_arrival_time': sum(inter_arrival_times) / len(inter_arrival_times) if inter_arrival_times else 0.0,
            'std_inter_arrival_time': (
                sum((x - sum(inter_arrival_times)/len(inter_arrival_times))**2 for x in inter_arrival_times) / len(inter_arrival_times)
            ) ** 0.5 if inter_arrival_times else 0.0,
        }

        output = {
            'packet_features': features,
            'statistical_features': statistical_features,
        }

        with open(self.output_file, 'w') as f:
            json.dump(output, f)

        return "Extracted enhanced packet features"

In [15]:
class CollectQoSMetricsTask(Task):
    def __init__(self, speedtest_output_file='speedtest_output.json', output_file='qos_metrics.json', *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.speedtest_output_file = speedtest_output_file
        self.output_file = output_file

    def run(self):
        import json

        with open(self.speedtest_output_file, 'r') as f:
            data = json.load(f)

        # Initialize QoS metrics with default values
        qos_metrics = {
            'download_speed_mbps': 0.0,
            'upload_speed_mbps': 0.0,
            'latency_ms': 0.0,
            'server': '',
        }

        # Extract download speed
        if 'download' in data:
            qos_metrics['download_speed_mbps'] = data['download'] / 1e6  # Convert bps to Mbps

        # Extract upload speed
        if 'upload' in data:
            qos_metrics['upload_speed_mbps'] = data['upload'] / 1e6  # Convert bps to Mbps

        # Extract latency
        if 'ping' in data:
            qos_metrics['latency_ms'] = data['ping']

        # Extract server info
        if 'server' in data and 'sponsor' in data['server']:
            qos_metrics['server'] = data['server']['sponsor']

        with open(self.output_file, 'w') as f:
            json.dump(qos_metrics, f)

        return "Collected enhanced QoS metrics"


In [ ]:
num_runs = 20
results_list = []

for i in range(num_runs):
    unique_id = f'run_{i}_{int(time.time())}'
    run_number = i  
    unique_id = f'run_{run_number}_{int(time.time())}'
    pipeline = (
        Pipeline()
        .then(ShellCommand(command='apt-get update && apt-get install -y tshark curl'))
        .then(ShellCommand(command='pip3 install pyshark speedtest-cli'))
        .then(StartCapture(filepath=f'capture_{unique_id}.pcap', arguments=['-i', 'any'], name='StartCapture'))
        .then(ShellCommand(command=f'speedtest-cli --json > speedtest_output_{unique_id}.json'))
        .then(StopCapture(start_capture_task_name='StartCapture'))
        .then(ExtractInitialPacketFeaturesTask(
            pcap_file=f'capture_{unique_id}.pcap',
            output_file=f'features_{unique_id}.json',
            packet_count=10,
            name='ExtractFeatures'
        ))
        .then(CollectQoSMetricsTask(
            speedtest_output_file=f'speedtest_output_{unique_id}.json',
            output_file=f'qos_metrics_{unique_id}.json',
            name='CollectQoS'
        ))
        .then(UploadToFileIO(filepath=f'features_{unique_id}.json', expires='14d', name='UploadFeatures'))
        .then(UploadToFileIO(filepath=f'qos_metrics_{unique_id}.json', expires='14d', name='UploadQoSMetrics'))
    )

    experiment_label = f'QoS_Metrics_Collection_experiment_{unique_id}'
    info = execute_pipeline(pipeline, working_nodes, experiment_label)
    
    for report in info.execution_result:
        if report.result is None:
            continue
        result, log = report.result
        if is_successful(result):
            data = result.unwrap()
            results_list.append({'unique_id': unique_id, 'data': data})
    time.sleep(60)  # To wait for network variability

ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED
ExperimentStatus.PREPARING
ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentS

In [50]:
features_list = []
qos_metrics_list = []

for result_entry in results_list:
    unique_id = result_entry['unique_id']
    data = result_entry['data']
    
    features_url = data['UploadFeatures'][0].unwrap().split('"link":"')[1].split('"')[0]
    qos_metrics_url = data['UploadQoSMetrics'][0].unwrap().split('"link":"')[1].split('"')[0]
    
    features_response = requests.get(features_url)
    features_data = features_response.json()
    features_data['unique_id'] = unique_id  
    features_list.append(features_data)
    
    qos_metrics_response = requests.get(qos_metrics_url)
    qos_metrics_data = qos_metrics_response.json()
    qos_metrics_data['unique_id'] = unique_id  
    qos_metrics_list.append(qos_metrics_data)

In [54]:

features_df = pd.json_normalize(
    features_list,
    record_path='packet_features',  
    meta=[
        'unique_id',
        ['statistical_features', 'mean_packet_size'],
        ['statistical_features', 'std_packet_size'],
        ['statistical_features', 'mean_inter_arrival_time'],
        ['statistical_features', 'std_inter_arrival_time']
    ],
    errors='ignore'
)
qos_metrics_df = pd.DataFrame(qos_metrics_list)
dataset = pd.merge(features_df, qos_metrics_df, on='unique_id')
categorical_columns = ['protocol', 'server']

for col in categorical_columns:
    dataset[col] = dataset[col].fillna('Unknown')

dataset = pd.get_dummies(dataset, columns=categorical_columns)
display(dataset)
X = dataset.drop(columns=['unique_id'])
y = dataset['download_speed_mbps'] # target variable
X = X.drop(columns=['download_speed_mbps'])
X = X.apply(pd.to_numeric, errors='coerce')
X.fillna(X.mean(), inplace=True)
X = X.loc[:, X.var() != 0]
X = X.dropna(axis=1, how='all')


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

if X_train.shape[0] < 2:
    raise ValueError("Not enough training samples to scale. Collect more data.")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

,packet_size,timestamp,inter_arrival_time,src_port,dst_port,sequence_number,ack_number,window_size,tcp_flags,unique_id,...,protocol_DNS,protocol_HTTP,protocol_ICMPV6,protocol_TCP,protocol_XML,server_Coastside.Net,server_Comcast,server_NETGEAR Inc.,server_Ridge Wireless,server_SiteHUB Agency
0,72,1.733479e+09,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,run_0_1733479045,...,False,False,True,False,False,True,False,False,False,False
1,79,1.733479e+09,0.283149,NaN,NaN,NaN,NaN,NaN,NaN,run_0_1733479045,...,True,False,False,False,False,True,False,False,False,False
2,79,1.733479e+09,0.000007,NaN,NaN,NaN,NaN,NaN,NaN,run_0_1733479045,...,True,False,False,False,False,True,False,False,False,False
3,158,1.733479e+09,0.004228,NaN,NaN,NaN,NaN,NaN,NaN,run_0_1733479045,...,True,False,False,False,False,True,False,False,False,False
4,184,1.733479e+09,0.000593,NaN,NaN,NaN,NaN,NaN,NaN,run_0_1733479045,...,True,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,76,1.733484e+09,0.001194,80.0,52886.0,0.0,1.0,65535.0,0x0012,run_19_1733483812,...,False,False,False,True,False,False,False,False,True,False
156,68,1.733484e+09,0.000020,52886.0,80.0,1.0,1.0,512.0,0x0010,run_19_1733483812,...,False,False,False,True,False,False,False,False,True,False
157,372,1.733484e+09,0.000061,52886.0,80.0,1.0,1.0,512.0,0x0018,run_19_1733483812,...,False,True,False,False,False,False,False,False,True,False
158,68,1.733484e+09,0.001138,80.0,52886.0,1.0,305.0,9.0,0x0010,run_19_1733483812,...,False,False,False,True,False,False,False,False,True,False


In [ ]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class QoSPredictor(nn.Module):
    def __init__(self, input_size):
        super(QoSPredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
   
    def forward(self, x):
        return self.model(x)

# Instantiate the model
input_size = X_train_tensor.shape[1]
model = QoSPredictor(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 1000
for epoch in range(epochs):
    model.train()  
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

# Evaluate on test set
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f"Test Loss: {test_loss.item():.4f}")


Epoch 1/1000, Loss: 173674.5703
Epoch 2/1000, Loss: 173433.4492
Epoch 3/1000, Loss: 173156.4336
Epoch 4/1000, Loss: 172830.7656
Epoch 5/1000, Loss: 172441.6680
Epoch 6/1000, Loss: 171982.9531
Epoch 7/1000, Loss: 171410.9453
Epoch 8/1000, Loss: 170751.9805
Epoch 9/1000, Loss: 169953.1250
Epoch 10/1000, Loss: 169043.9297
Epoch 11/1000, Loss: 167916.5469
Epoch 12/1000, Loss: 166655.4102
Epoch 13/1000, Loss: 165182.3281
Epoch 14/1000, Loss: 163429.4648
Epoch 15/1000, Loss: 161482.9062
Epoch 16/1000, Loss: 159187.0078
Epoch 17/1000, Loss: 156643.7930
Epoch 18/1000, Loss: 153823.1719
Epoch 19/1000, Loss: 150583.9727
Epoch 20/1000, Loss: 147046.3359
Epoch 21/1000, Loss: 143123.4648
Epoch 22/1000, Loss: 138873.5117
Epoch 23/1000, Loss: 134265.0898
Epoch 24/1000, Loss: 129186.9805
Epoch 25/1000, Loss: 123779.4512
Epoch 26/1000, Loss: 118216.0723
Epoch 27/1000, Loss: 112202.7422
Epoch 28/1000, Loss: 105987.4590
Epoch 29/1000, Loss: 99388.5293
Epoch 30/1000, Loss: 92735.0918
Epoch 31/1000, Loss: 

In [ ]:
class NNRegressor:
    def __init__(self, model, device='cpu'):
        self.model = model.to(device)
        self.device = device
        self.model.eval()
    
    def predict(self, X):
        X_tensor = torch.tensor(X, dtype=torch.float32, device=self.device)
        with torch.no_grad():
            outputs = self.model(X_tensor).cpu().numpy()
        return outputs.ravel()

regressor_expert = NNRegressor(model)

trustee = RegressionTrustee(expert=regressor_expert)
trustee.fit(X_train, y_train, num_samples=len(X_train)//2, num_iter=20, train_size=0.9)

# Explain the model using a decision tree
_, dt, _, score = trustee.explain()
print(f"Training score of pruned DT: {score}")

# Evaluate fidelity
dt_y_pred = dt.predict(X_train)
print("Model explanation global fidelity report:")
print("MSE:", metrics.mean_squared_error(regressor_expert.predict(X_train), dt_y_pred))
print("MAE:", metrics.mean_absolute_error(regressor_expert.predict(X_train), dt_y_pred))

# Plot the tree 
feature_names = [f"feat_{i}" for i in range(X_train.shape[1])]
fig = plt.figure(figsize=(25,20))
plot_tree(dt, feature_names=feature_names, filled=True, max_depth=7)
plt.show()
